# CS5785 final - approach 8
### This file contains additional experiments (approach 8) performed on the project. ***read the warnings below before you run the code!!***

# **WARNING**: Due to the large amount of data generated, this file is only suitable to run in Google Colab or other similar services. Running this file on your local machine is strongly discouraged and could cause unexpected and/or undesirable consequences.

In [0]:
#!pip install gensim

In [1]:
#!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2019-12-07 22:41:46--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.0.182
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.0.182|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G   126MB/s    in 17s     

2019-12-07 22:42:04 (90.4 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [0]:
import os
import csv
import random
#import gensim
import numpy as np

num_train = 8000
num_dev = 2000
num_test = 2000
split_idx = list(range(num_train + num_dev))
random.shuffle(split_idx)

In [2]:
#import saved .npy matrices from google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
x_train = np.load('/content/drive/My Drive/amlfinal/xtrain.npy')
x_test = np.load('/content/drive/My Drive/amlfinal/xtest.npy')
x_dev = np.load('/content/drive/My Drive/amlfinal/xdev.npy')
y_train = np.load('/content/drive/My Drive/amlfinal/ytrainfull.npy')
y_test = np.load('/content/drive/My Drive/amlfinal/ytestfull.npy')
y_dev = np.load('/content/drive/My Drive/amlfinal/ydevfull.npy')

In [8]:
import random

augmented_train = []

for i in range(8000):
    true_row = np.concatenate((x_train[i], y_train[i], 1), axis=None)
    random_1 = i
    while random_1 == i:
        random_1 = random.randint(0, 7999)
    
    random_2 = i
    while random_2 == i:
        random_2 = random.randint(0, 7999)

    random_3 = i
    while random_3 == i:
        random_3 = random.randint(0, 7999)
    
    false_row_1 = np.concatenate((x_train[i], y_train[random_1], 0), axis=None)
    false_row_2 = np.concatenate((x_train[i], y_train[random_2], 0), axis=None)
    false_row_3 = np.concatenate((x_train[i], y_train[random_3], 0), axis=None)
    
    augmented_train.append(true_row)    
    augmented_train.append(false_row_1)
    augmented_train.append(false_row_2)
    augmented_train.append(false_row_3)

augmented_train = np.asarray(augmented_train)
augmented_train_x = augmented_train[:,:-1]
augmented_train_y = augmented_train[:,-1]

print(augmented_train_x.shape, augmented_train_y.shape)

(32000, 1300) (32000,)


# **WARNING**: Due to the large amount of data generated, all sections below are only suitable to run in Google Colab or other similar services. Running this file on your local machine is strongly discouraged and could cause unexpected and/or undesirable consequences.

In [0]:
#generating validation set
augmented_val = [[0] * 1301] * (400*2000) 

In [0]:
counter = 0
for i in range(0, 400, 1):
    for j in range(2000):
        if i == j:
            true_row = np.concatenate((x_dev[i], y_dev[j], 1), axis=None)
            augmented_val[counter] = true_row
        else:
            false_row = np.concatenate((x_dev[i], y_dev[j], 0), axis=None)
            augmented_val[counter] = false_row
        counter += 1

In [0]:
aug_val = np.asarray(augmented_val)
augmented_val_x = aug_val[:,:-1]
augmented_val_y = aug_val[:,-1]

In [13]:
print(augmented_val_x.shape, augmented_val_y.shape)

(800000, 1300) (800000,)


In [0]:
#releasing memory
augmented_val = []
aug_val = []

In [0]:
#data scaling with MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train = scaler.fit_transform(augmented_train_x)
augmented_train_x = []

In [0]:
X_val = scaler.fit_transform(augmented_val_x)
augmented_val_x = []

In [61]:
X_train.shape

(32000, 1300)

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

rf_params = {'max_depth': [50, 100, 200], 'n_estimators': [50, 100, 250]}
rf_model = GridSearchCV(RandomForestClassifier(), rf_params, cv=5, n_jobs=-1, verbose=10)
rf_model.fit(X_train, augmented_train_y)
rf_model_best = rf_model.best_estimator_
print(rf_model_best)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  1.3min
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  9.0min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 13.4min
[Parallel(n_jobs=-1)]: Done  43 out of  45 | elapsed: 18.3min remaining:   51.2s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed: 20.2min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=100, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=250,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


In [16]:
y_valid_pred = rf_model.predict_proba(augmented_val_x)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    2.9s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    4.9s
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:    7.6s
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:   10.8s
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:   14.2s
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   17.8s
[Parallel(n_jobs=4)]: Done  53 tasks      | elapsed:   22.7s
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:   26.9s
[Parallel(n_jobs=4)]: Done  77 tasks      | elapsed:   32.2s
[Parallel(n_jobs=4)]: Done  90 tasks      | elapsed:   37.3s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:   41.7s finished


In [21]:
y_val_pred = y_valid_pred[:,0]
y_val_pred = y_val_pred.reshape((400, 2000))
y_val_pred.shape

(400, 2000)

In [25]:
dev_scores = []
dev_pos_list = []
counter = 400

for idx, item in enumerate(y_val_pred):
  sorted_pred = list(np.argsort(item))
  true_position = sorted_pred.index(counter)
  dev_pos_list.append(true_position)
  if true_position < 20:
    dev_scores.append(1 / (true_position + 1))
  else:
    dev_scores.append(0.0)
  counter += 1

print("Development MAP@20:", np.mean(dev_scores))
print("Mean index of true image", np.mean(dev_pos_list))
print("Median index of true image", np.median(dev_pos_list))


Development MAP@20: 0.08731495892738153
Mean index of true image 84.365
Median index of true image 34.5
